In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
cellrebel = pd.read_feather('Source Data/consolidate_file.feather')
# cellrebel = pd.read_csv('Source Data/cellrebel_20230227-20230310.csv', low_memory=False, parse_dates=[1])
# grid = gpd.read_file('Source Data/JORR - Polygon.geojson')
grid = pd.read_csv('Grid Folder/Bad RSRP Performance_grid200_20 Spot.csv')

In [3]:
grid = gpd.GeoDataFrame(grid, geometry=gpd.points_from_xy(grid['Long Grid'], grid['Lat Grid']), crs=4326)

In [4]:
grid

,Lat Grid,Long Grid,Enodebid Ci,Kel Des,Avg Rsrp Serving,Total Samples,geometry
0,-6.219935,106.881552,440842_5,CIPINANG BESAR UTARA,-106.434602,2890,POINT (106.88155 -6.21994)
1,-6.311563,106.890535,440619_3,CEGER,-105.539606,8067,POINT (106.89053 -6.31156)
2,-6.273834,106.886942,440617_9,HALIM PERDANA KUSUMAH,-105.656495,3441,POINT (106.88694 -6.27383)
3,-6.306173,106.917484,440587_6,SETU,-105.010822,3419,POINT (106.91748 -6.30617)
4,-6.354682,106.881552,440410_8,CIBUBUR,-105.269262,3699,POINT (106.88155 -6.35468)
5,-6.338513,106.908501,440314_4,CILANGKAP,-106.812453,5300,POINT (106.90850 -6.33851)
6,-6.320546,106.877958,440311_2,CIRACAS,-105.416759,3604,POINT (106.87796 -6.32055)
7,-6.349292,106.904908,440260_4,PONDOK RANGGON,-108.310221,3346,POINT (106.90491 -6.34929)
8,-6.324140,106.894128,440237_5,CIPAYUNG,-105.664440,3743,POINT (106.89413 -6.32414)
9,-6.351089,106.894128,440139_1,MUNJUL,-105.889796,3920,POINT (106.89413 -6.35109)


In [ ]:
cellrebel['mobile_operator'].replace('PT Telekomunikasi Indonesia', 'Telkomsel', inplace=True)
cellrebel['mobile_operator'].replace('3', 'Indosat Ooredoo', inplace=True)

In [ ]:
cellrebel = cellrebel[cellrebel['mobile_operator'].str.contains('XL Axiata|Indosat Ooredoo|Telkomsel|Smartfren')== True ]
cellrebel = cellrebel[cellrebel['created_date'].astype(str).str.contains('2023-02-25|2023-02-26|2023-03-04|2023-03-05|2023-03-11|2023-03-12|2023-03-18|2023-03-19')==False]

In [ ]:
cellrebel['created_date'] = pd.to_datetime(cellrebel['created_date'])

In [ ]:
hour_filter = (cellrebel['created_date'].dt.hour == 7) | (cellrebel['created_date'].dt.hour == 8) | (cellrebel['created_date'].dt.hour == 9)| (cellrebel['created_date'].dt.hour == 18)

cellrebel = cellrebel[hour_filter]

In [ ]:
cellrebel_cov = cellrebel[cellrebel['reference_signal_received_power'] < 0]
cellrebel_cap = cellrebel[cellrebel['dl_throughput'].astype(int) > 0]

In [ ]:
cellrebel_cov['reference_signal_received_power'].sort_values()

In [ ]:
cellrebel_gpd_cov = gpd.GeoDataFrame(cellrebel_cov, geometry=gpd.points_from_xy(cellrebel_cov.longitude, cellrebel_cov.latitude), crs=4326)
cellrebel_gpd_cap = gpd.GeoDataFrame(cellrebel_cap, geometry=gpd.points_from_xy(cellrebel_cap.longitude, cellrebel_cap.latitude), crs=4326)
grid = grid.to_crs(4326)
grid['geometry1'] = grid['geometry']

In [ ]:
points_within_cov = gpd.sjoin(left_df= cellrebel_gpd_cov,right_df=grid, how='left', predicate='within')
points_within_cap = gpd.sjoin(left_df= cellrebel_gpd_cap,right_df=grid, how='left', predicate='within')

In [ ]:
points_within_cap['geometry1'].value_counts()

In [ ]:
pivot_cov = pd.pivot_table(points_within_cov, index= ['poly_name', 'index_right'], columns=['mobile_operator'], values=['reference_signal_received_power'], aggfunc = np.mean)
pivot_cap = pd.pivot_table(points_within_cap, index= ['poly_name', 'index_right'], columns=['mobile_operator'], values=['dl_throughput'], aggfunc = lambda x: x.quantile(0.1))

In [ ]:
pivot_cov.fillna(-150, inplace=True)
pivot_cap.fillna(0, inplace=True)

In [ ]:
pivot_cov.reset_index(inplace=True, col_level=1, allow_duplicates=False)
pivot_cap.reset_index(inplace=True, col_level=1, allow_duplicates=False)

In [ ]:
pivot_cov = pivot_cov.droplevel(level=0, axis=1)
pivot_cap = pivot_cap.droplevel(level=0, axis=1)

In [ ]:
pivot_cov = pivot_cov.merge(points_within_cov[['index_right', 'geometry1']], how='left', on='index_right')
pivot_cap = pivot_cap.merge(points_within_cap[['index_right', 'geometry1']], how='left', on='index_right')

In [ ]:
pivot_cov.drop_duplicates(ignore_index=True, inplace=True)
pivot_cap.drop_duplicates(ignore_index=True, inplace=True)

In [ ]:
pivot_cov.to_csv('result/p10_poi_coverage_result.csv')
pivot_cap.to_csv('result/p10_poi_capacity_result.csv')
cellrebel.to_csv('processed_data.csv')